In [1]:
import sys
import glob
import os
import cv2
import numpy as np
from __future__ import absolute_import

import tensorflow as tf
#print ("tensorflow v:" + tf.__version__)


from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
#!conda install --yes --prefix {sys.prefix} opencv
#!{sys.executable} -m pip install opencv-contrib-python
#conda install -c conda-forge opencv

Using TensorFlow backend.


import sys
!conda install --yes --prefix {sys.prefix} keras


In [2]:

#images = []

skinny_j = []
straight_j = []
bootc = []
def find(obj, l, y):
    dir1 = "/Users/Simron/ds/examples/pants/proj/images/" + obj
   
    
    #for x in os.walk(dir1):
            #print x
    m_dir= glob.glob(dir1 + "/*/*jpg")
    for files in m_dir:
            image = cv2.imread(files, 1)
            #print image.shape
            
            #print image.shape
            #asp = 100.0 / image.shape[1]
            #dim = (100, int(image.shape[0]*asp))
            
            #resize = cv2.resize(image, dim, interpolation = cv2.INTER_AREA)
            #print resize.shape
            #l.append(resize)
            #tup = image.append(y)
            l.append(image)
    l = np.array(l)
    #print ("shape:", l.shape)
    return l
    
bootc = find("bc", bootc, 0)
#print bootc[0].shape
skinny_j = find("skinny", skinny_j, 1)
straight_j = find("straight", straight_j, 2)

y_bc = np.full((bootc.shape[0], 1), 0)
y_skj = np.full((skinny_j.shape[0], 1), 1)
y_sj = np.full((straight_j.shape[0], 1), 2)



All of the images come of different shapes and sizes so rather than downsampling certain images and risking the loss of data, we will resize and pad certain images and compare which is better.

In [3]:
#730 x 480

def resize(l, size):
    ret_l = []   
    count = 0
    for i in range(len(l)):        
        o_size = l[i].shape[:2]

        ratio = float(size)/max(o_size)
        n_size = tuple([int(x*ratio) for x in o_size])

        im = l[i]
        im = cv2.resize(im, (n_size[1], n_size[0]))

        delta_w = size - n_size[1]
        delta_h = size - n_size[0]
        top, bottom = delta_h//2, delta_h-(delta_h//2)
        left, right = delta_w//2, delta_w-(delta_w//2)
        color = [0, 0, 0]
        new_im = cv2.copyMakeBorder(im, top, bottom, left, right, cv2.BORDER_CONSTANT, value=color)
        #print new_im.shape
        #tup = (new_im, l[i])
        ret_l.append(new_im)
    #for e in np.array(ret_l):
            #print e.shape
    return np.array(ret_l)

In [4]:
bootc_r = resize(bootc, 730)
#print bootc_r[0].shape

In [5]:
skinny_r = resize(skinny_j, 730)
straight_r = resize(straight_j, 730)

In [6]:
model = Sequential()
#Convolution
model.add(Conv2D(32, (3, 3), input_shape = (730, 730, 3), activation = 'relu'))

#Pooling
model.add(MaxPooling2D(pool_size = (2, 2)))


# Adding a second convolutional layer
model.add(Conv2D(32, (3, 3), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))


# Flattening
model.add(Flatten())

# Full connection
model.add(Dense(units = 128, activation = 'relu'))
model.add(Dense(units = 1, activation = 'sigmoid'))

# Compiling the CNN
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])


In [7]:


X = np.vstack([bootc_r, skinny_r, straight_r])
y = np.vstack([y_bc, y_skj, y_sj])

X2 = X
y2 = y

#print ("x shape:" + X.shape)
#print ("y shape:" + y.shape)


In [8]:
X2 = X
y2 = y
data = zip(X2, y2)
#np.random.shuffle(data)

In [ ]:
print len(data)
data1 = np.array(data)
X2 = np.array([])
y2 = np.array([])
count = 0
for i, j in data1:
    X2 = np.append(X2, i)
    y2 = np.append(y2, j)
    count = count+1
#X2, y2 = [[i,j] for i,j in data1]
#print data1[0][1].shape
#X2, y2 = data1

1675


In [ ]:
print count
#print X2.shape
print y2
#X = data[:][0]
#print(data1[:][1])
#y = data1[:][:][0]

#print y[1].shape

X2 = X
data = np.array([])
i = 0
#for e in X2:
   #np.concatenate(e, y[i])
    #i = i+1
    #break
np.concatenate(X2, y)
print X2[0].shape

#data = np.hstack([X, y.T])
#data = X + y.T
i = 0
data = np.array([])
for e in X:
    tup = (e, y[i])
    i = i+1
    data = np.append(data, tup)
#X2 = np.array([])
np.random.shuffle(data)
for e in data:
    X = np.append(X,e[0])
    y = np.append(y, e[1])
    
print y[0]
print y[1]


In [ ]:
print (X.shape)
print (y.shape)


Now we shall normalize the data by subtracting the mean of the samples from each sample. Additionally we will divide by the standard deviation of each pixel. Subtracting the mean centers the input to 0, and dividing by the standard deviation makes any scaled feature value the number of standard deviations away from the mean.

In [9]:
from sklearn.preprocessing import StandardScaler

X = X - np.mean(X, axis=0)
print "hello1"
X = X/np.std(X, axis=0)
print ("hello2")

hello1


ValueError: Found array with dim 4. StandardScaler expected <= 2.

In [10]:
from sklearn.model_selection import cross_val_score

cv_scores = cross_val_score(model, X, y, cv=5)

cv_scores.mean()

TypeError: If no scoring is specified, the estimator passed should have a 'score' method. The estimator <keras.engine.sequential.Sequential object at 0x1a37f2f590> does not.

## LOOCV

y_pred = []

for i in range(len(X)):
    abi = range(0, i) + range(i+1, len(X))
    X_train = X[abi]
    y_train = y[abi]
    
    model.fit(X_train, y_train, shuffle=True, epochs=150)
    X_test = X[i]
    y_test = y[i]
    y_act = model.predict(X_test)
    y_pred.append(y_act)
accuracy = 0
for i in range(len(X)):
    if (y_pred[i] == y[i]):
        accuracy = accuracy + 1
        
print ("accuracy : %d / %d" % (accuracy, len(X)))